Import

In [2]:
# Import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

Part 1 - Browser Automation with Selenium

In [3]:

options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)
driver.get("http:www.imdb.com")
driver.implicitly_wait(5)

# search = driver.find_element(By.XPATH, "//input[@id='suggestion-search']")
# search.send_keys("Godfather")

# All Dropdown
all_dropdown = driver.find_element(By.XPATH, "//span[@class='ipc-btn__text'][normalize-space()='All']" )
# driver.implicitly_wait(5)
all_dropdown.click()


advanced_search = driver.find_element(By.XPATH, "//a[@class='ipc-list__item searchCatSelector__item']//span[@role='presentation'][normalize-space()='']")
advanced_search.click()

advanced_title_search = driver.find_element(By.XPATH,'//*[@id="main"]/div[2]/div[1]/a')
advanced_title_search.click()

title_type_1 = driver.find_element(By.ID,'title_type-1')
title_type_1.click()

# title_type_3 = driver.find_element(By.ID,'title_type-3')
# title_type_3.click()

release_date_min = driver.find_element(By.NAME, 'release_date-min')
release_date_min.send_keys(1980)

release_date_max = driver.find_element(By.NAME, 'release_date-max')
release_date_max.send_keys(2023)

rating_min = driver.find_element(By.NAME,'user_rating-min')
rating_min.click()
rating_min_dropdown = Select(rating_min)
rating_min_dropdown.select_by_visible_text('6.0')

rating_max = driver.find_element(By.NAME,'user_rating-max')
rating_max.click()
rating_max_dropdown = Select(rating_max)
rating_max_dropdown.select_by_visible_text('10')

# genre_1 = driver.find_element(By.ID,'genres-6')
# genre_1.click()
genre_2 = driver.find_element(By.ID,'genres-24')
genre_2.click()
# genre_3 = driver.find_element(By.ID,'genres-26')
# genre_3.click()

title_group_1 = driver.find_element(By.ID,'groups-7')
title_group_1.click()

language = driver.find_element(By.NAME,'languages')
language_dropdown = Select(language)
language_dropdown.select_by_visible_text('English')

page_count = driver.find_element(By.ID,'search-count')
page_count_dropdown = Select(page_count)
page_count_dropdown.select_by_visible_text('100 per page')

submit = driver.find_element(By.XPATH,'(//button[@type="submit"])[2]')
submit.click()

current_url = driver.current_url

sort_by_rating = driver.find_element(By.XPATH,"//a[normalize-space()='User Rating']")
sort_by_rating.click()

Part 2 - Data Extraction with Beautiful Soup

In [4]:
response = requests.get(current_url)
response

<Response [200]>

In [21]:
# Soup Object
soup = bs(response.content, 'html.parser')

# All movie list
list_items = soup.find_all('div',{'class':'lister-item'})

In [6]:
list_items = soup.find_all('div',{'class':'lister-item'})
len(list_items)

100

In [7]:
title = list_items[0].find('h3').find('a').get_text().strip()
title

'The Dark Knight'

In [8]:
year = list_items[0].find('h3').find('span', {'class':'lister-item-year'}).get_text().strip()[1:-1]
year

'2008'

In [9]:
runtime = list_items[0].find('p').find('span',{'class':'runtime'}).get_text().strip()
runtime

'152 min'

In [10]:
genre = list_items[0].find('span', {'class':'genre'}).get_text().strip()
genre

'Action, Crime, Drama'

In [11]:
rating = list_items[0].find('div',{'class':'ratings-imdb-rating'}).get_text().strip()
rating

'9.0'

List Comprehension

In [20]:
title = [result.find('h3').find('a').get_text().strip() for result in list_items]
year = [result.find('h3').find('span', {'class':'lister-item-year'}).get_text().strip()[1:-1] for result in list_items]
runtime = [result.find('p').find('span',{'class':'runtime'}).get_text().strip() for result in list_items]
genre = [result.find('span', {'class':'genre'}).get_text().strip() for result in list_items]
rating = [result.find('div',{'class':'ratings-imdb-rating'}).get_text().strip() for result in list_items]

In [14]:
title = [result.find('h3').find('a').get_text().strip() for result in list_items]
len(title)

100

In [15]:
year = [result.find('h3').find('span', {'class':'lister-item-year'}).get_text().strip()[1:-1] for result in list_items]
len(year)

100

In [17]:
runtime = [result.find('p').find('span',{'class':'runtime'}).get_text().strip() for result in list_items]
len(runtime)

100

In [18]:
genre = [result.find('span', {'class':'genre'}).get_text().strip() for result in list_items]
len(genre)

100

In [19]:
rating = [result.find('div',{'class':'ratings-imdb-rating'}).get_text().strip() for result in list_items]
len(rating)

100

In [82]:
# All fields
title = [result.find('h3').find('a').get_text().strip() for result in list_items]
year = [result.find('h3').find('span', {'class':'lister-item-year'}).get_text().replace('(','').replace(')','').replace('I','') for result in list_items]
# fix int stored as text
# year = [int(value) for value in year]
runtime = [result.find('p').find('span',{'class':'runtime'}).get_text().strip() for result in list_items]
genre = [result.find('span', {'class':'genre'}).get_text().strip() for result in list_items]
rating = [result.find('div',{'class':'ratings-imdb-rating'}).get_text().strip() for result in list_items]
# fix int stored as text
# rating = [float(value) for value in rating]
# Round rating to x.0 format
# rating = [f"{value:.1f}" for value in rating]

In [86]:
# Dataframe
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df = pd.DataFrame({'Title':title, 'Year':year, 'Duration':runtime, 'Genre':genre, 'Rating':rating})

# Convert the 'Value' column to numeric dtype
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')


df.to_excel('movie_list.xlsx', index=False)

df

,Title,Year,Duration,Genre,Rating
0,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0
1,Inception,2010,148 min,"Action, Adventure, Sci-Fi",8.8
2,Tenet,2020,150 min,"Action, Sci-Fi, Thriller",7.3
3,Dunkirk,2017,106 min,"Action, Drama, History",7.8
4,The Batman,2022,176 min,"Action, Crime, Drama",7.8
5,The Prestige,2006,130 min,"Drama, Mystery, Sci-Fi",8.5
6,Memento,2000,113 min,"Mystery, Thriller",8.4
7,Blade Runner 2049,2017,164 min,"Action, Drama, Mystery",8.0
8,Jurassic Park,1993,127 min,"Action, Adventure, Sci-Fi",8.2
9,The Departed,2006,151 min,"Crime, Drama, Thriller",8.5
